# Sistema de recomendação de filmes

<img src='background.png' width ='950' heigth = '30'/>

Aqui apresentamos a construção de um sistema de recomendação; ferramenta muito útil em diversos segmentos do mercado. Neste exemplo, utilizaremos este recurso para a recomendação de filmes, tendo como principal referência o trabalho desenvolvido pela [Ibtesan Ahmed](https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system/data) mas apresentaremos outras referências ao longo da leitura como material complementar. 

Este documento contém os seguintes itens:
1. Filtro por avaliação
2. Filtro por popularidade
3. Filtro baseado em conteúdo

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')

Carregadas as bases de dados, iremos agora unir cada uma delas a fim de trabalharmos com apenas um dataframe. Para tanto, usaremos o método **merge** e escolheremos a coluna de **id** como 'ponto' de união dos dataframes.

In [3]:
df1.columns = ['id','tittle','cast','crew']
df2 = df2.merge(df1, on='id')

### Filtro baseado em avaliações

Agora desejamos estabelecer critérios para que os filmes sejam recomendados, i.e., estabelecer uma faixa a partir das quais  possamos decidir se o filme pode ou não ser recomendado futuramente. 

Então, separamos inicialmente dois critérios que serão a média dos votos **(C)** e nota mínina para recomendação **(m)**. Vejamos:

In [4]:
C= df2['vote_average'].mean() 
C

6.092171559442011

Com isso, temos que a classificação média para todos os filmes é de aproximadamente 6 e vamos agora determinar um valor mínimo de votos necessários para que o filme venha a ser listado como futura recomendação, afinal não desejamos recomendar filmes que não foram bem avaliados. Chamaremos de **m*** e usaremos o percentil 90 como ponto de corte.

In [6]:
m= df2['vote_count'].quantile(0.9) #voto mínimo
m

1838.4000000000015

Vejamos os filmes que atendem aos requisitos pré-estabelecidos. Estes são os filmes que serão recomendados.

In [8]:
#Filmes que entram na lista de recomendação
q_movies = df2.copy().loc[df2['vote_count'] >= m]
q_movies.shape

(481, 23)

Com isso, vemos que temos 481 filmes que podem ser recomendados aos usuários. Agora vamos definir uma métrica para cada filme qualificado. Para isso, definiremos a função, **weighted_rating()** e definiremos uma nova pontuação.

In [9]:
#Função que calcula nota ponderada dos filmes
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calcula baseando-se na fórmula do IMDB
    return (v/(v+m) * R) + (m/(m+v) * C)

A seguir vamos definir um novo score, que é calculado a partir da função anterior. Feito isso, vamos ordenar os filmes segundo o score e exibir.

In [10]:
#Definindo novo score para os filmes
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

#Ordenando os filmes com base no score
q_movies = q_movies.sort_values('score', ascending=False)

#Exibindo a dataframe
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
1881,The Shawshank Redemption,8205,8.5,8.059258
662,Fight Club,9413,8.3,7.939256
65,The Dark Knight,12002,8.2,7.920020
3232,Pulp Fiction,8428,8.3,7.904645
96,Inception,13752,8.1,7.863239
3337,The Godfather,5893,8.4,7.851236
95,Interstellar,10867,8.1,7.809479
809,Forrest Gump,7927,8.2,7.803188
329,The Lord of the Rings: The Return of the King,8064,8.1,7.727243
1990,The Empire Strikes Back,5879,8.2,7.697884


### Filtro baseado na popularidade

Aqui apresentamos de forma breve um exemplo que sistema de recomendação a partir de uma característica, neste caso, a popularidade do filme.

In [11]:
#Filmes populares 
pop= df2.sort_values('popularity', ascending=False)

In [12]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.barh(pop['title'].head(6),pop['popularity'].head(6), align='center',
        color='skyblue')
plt.gca().invert_yaxis()
plt.xlabel("Popularity")
plt.title("Filmes populares")
plt.show()

<Figure size 1200x400 with 1 Axes>

Este gráfico nos fornece possíveis recomendações para todos os usuários dada a popularidade dos filmes no entanto,  este resultado não é sensível aos interesses e gostos dos usuários. Então precisamos estabelecer outro tipo de sistema, que apresentaremos a seguir.

### Filtros baseados em conteúdo

A filtragem __Baseada em Conteúdo__ avalia aspectos como visão geral, elenco, equipe, palavra-chave, slogan para encontrar semelhança com outros filmes e poder sugerir ao usuário que tenha fornecido avaliações positivas anteriormente a filmes semelhantes.

Um bom passo inicial para buscar filmes semelhantes é avaliar a coluna **overview**, uma vez que, traz uma descrição breve do filme de modo que podemos de forma rápida filtrar filmes de segmentos/histórias semelhantes para futuras recomendações.

Para isso é preciso ter uma breve compreensão sobre **processamento de texto** caso não tenha recomendamos a leitura das seguintes documentações: [material 1](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) e [material 2](https://scikit-learn.org/stable/modules/feature_extraction.html). Aqui, contaremos com a biblioteca **TfidVectorizer** que consiste em transformar dados arbitrários, como texto ou imagens, em recursos numéricos utilizáveis para aprendizado de máquina e nos auxiliará muito nessa etapa.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

O primeiro passo que daremos é definir um objeto vetorizador TF-IDF que remova todas as palavras de parada em inglês, como **'the', 'a'**, porque dentre outros motivos, estas palavras sempre ocorrem com bastante frequência e agregam pouca compreensão dos filmes. Em seguida iremos remover os missing-values atribuindo-lhes espaço.

In [14]:
tfidf = TfidfVectorizer(stop_words='english')
df2['overview'] = df2['overview'].fillna('')

A seguir iremos converter uma coleção de documentos brutos em uma matriz de recursos TF-IDF, de modo que pelo método **fit_transform** ele aprenda o vocabulário idf e retorna a matriz termo-documento. 

In [15]:
#Matriz TF-IDF necessária ajustando e transformando os dados
tfidf_matrix = tfidf.fit_transform(df2['overview'])
#tfidf_matrix.shape

(4803, 20978)

Note que pelo shape de $X$ vemos que mais de 20.000 palavras diferentes foram usadas para descrever os filmes que estamos trabalhando.

O próximo passo é calcular os scores de similaridade (este podem ser euclidiano, Pearson e cosseno) entre os filmes e para isso, usaremos a matriz TF-IDF. Note que, estamos trabalhando com matrizes sendo assim, é importante conhecer o conceito de **matriz de similaridade**, que nada mais é  uma matriz tal que os elementos medem as semelhanças entre pares de objetos.

Aqui usaremos a semelhança de cosseno para calcular uma quantidade numérica que denota a semelhança entre dois filmes. Como usamos o vetorizador TF-IDF, o cálculo do produto escalar fornece diretamente a pontuação de similaridade do cosseno. Assim, usaremos o **linear_kernel()** em vez de **cosine_similarities()**, para otimizar o tempo.

In [16]:
from sklearn.metrics.pairwise import linear_kernel

#Cálculo da semelhança de cosseno
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

Nosso objetivo agora é: _Dado o nome de um filme obter uma lista de filmes semelhantes a ele. No entanto, precisamos identificar o índice de um filme dado o título_. 

In [17]:
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()

Agora iremos definir algumas etapas para definir uma forma de recomendação. Para tanto, dado um título iremos obter o seu índice. Em seguida, iremos obter a lista de similaridade desse título do qual extrairemos os 10 elementos principais.

In [18]:
# Função que recebe o título do filme como entrada e produz lista de filmes semelhantes
def gera_recomend(titulo, cosine_sim = cosine_sim):
    # Obtem o índice do título dado
    idx = indices[titulo]

    # Obtem os scores de similaridade com o filme dado
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordena os scores de similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtem os 10 filmes mais similares pelo score
    sim_scores = sim_scores[1:11]

    # Obtem os índices dos filmes similares
    movie_indices = [i[0] for i in sim_scores]

    # Retorna 10 indicações
    return df2['title'].iloc[movie_indices]

In [19]:
gera_recomend('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title, dtype: object

Neste momento iremos buscar melhorias ao sistema de recomendação, usando por exemplo o gênero dos filmes, palavras-chave entre outras características.

In [20]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

In [21]:
# Obtenha o nome do diretor no recurso de equipe. Se o diretor não estiver, retorne NaN
def tome_diretor(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [22]:
# Retorne 3 principais da lista
def gere_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Se houver mais de 3 elementos retorne apenas três.
        if len(names) > 3:
            names = names[:3]
        return names

    #Retornar lista vazia se houver dados ausentes
    return []

In [23]:
# Define novos recursos
df2['director'] = df2['crew'].apply(tome_diretor)

features = ['cast', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(gere_list)

In [24]:
# Apresenta novos recursos do top 3
df2[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"


O próximo passo é converter os nomes em letras minúsculas e remover os espaços. Faremos isso para que nosso vetorizador não conte o Johnny de "Johnny Depp" e "Johnny Galecki" como o mesmo.

In [25]:
# Função para converter todas as strings em minúsculas
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Checa de existe diretor
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [26]:
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

Agora iremos criar nossa "sopa de metadados", que é uma string que contém todos os metadados que queremos alimentar para o nosso vetorizador (atores, diretor e palavras-chave).

In [27]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)

In [28]:
#não queremos reduzir o peso da presença de um ator/diretor
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])

In [29]:
# Calcula o score de similaridade com base no count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [30]:
# Redefina o índice do nosso DataFrame principal e construa o mapeamento reverso como antes
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

In [31]:
gera_recomend('Batman Begins', cosine_sim2)

3          The Dark Knight Rises
65               The Dark Knight
4638    Amidst the Devil's Wings
1196                The Prestige
1742              Brick Mansions
3603           Lone Wolf McQuade
982                Run All Night
3326              Black November
1503                      Takers
1986                      Faster
Name: title, dtype: object

Há outras técnicas que são usadas para criar listas de recomendação. Das técnicas aqui apresentadas consideramos o nosso objetivo alcançado de modo a obter uma lista de recomendação coerente.

Atualemente, estamos trabalhando na criação de uma aplicação em Streamlit que utiliza dessas técnicas para trazer recomendações ao usuário, tendo concluído esta aplicação atualizo com o link este arquivo. 